In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .config("spark.sql.autoBroadcastJoinThreshold", 0)
    .master("local[*]")
    .getOrCreate()
}

def sc = spark.sparkContext // Разница между def, val, lazy val

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@14181a8f
defined function sc

In [3]:
val accum = sc.longAccumulator("My Accumulator")

sc.parallelize(Array(1, 2, 3, 4)).foreach(x => accum.add(x))

accum.value

foreach at cmd2.sc:3

12 / 12

accum: org.apache.spark.util.LongAccumulator = LongAccumulator(id: 26, name: Some(My Accumulator), value: 10)
res2_2: java.lang.Long = 10L

In [11]:
val crimeFacts = spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("crime.csv")

crimeFacts.show

csv at cmd10.sc:5

1 / 1

csv at cmd10.sc:5

12 / 12

show at cmd10.sc:7

1 / 1

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One|       LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|           VANDALISM|     C11|   

crimeFacts: DataFrame = [INCIDENT_NUMBER: string, OFFENSE_CODE: int ... 15 more fields]

In [10]:
import spark.implicits._

crimeFacts.groupBy($"OFFENSE_CODE").count.filter($"OFFENSE_CODE" === 1402).explain(true)

== Parsed Logical Plan ==
'Filter ('OFFENSE_CODE = 1402)
+- Aggregate [OFFENSE_CODE#214], [OFFENSE_CODE#214, count(1) AS count#395L]
   +- Relation[INCIDENT_NUMBER#213,OFFENSE_CODE#214,OFFENSE_CODE_GROUP#215,OFFENSE_DESCRIPTION#216,DISTRICT#217,REPORTING_AREA#218,SHOOTING#219,OCCURRED_ON_DATE#220,YEAR#221,MONTH#222,DAY_OF_WEEK#223,HOUR#224,UCR_PART#225,STREET#226,Lat#227,Long#228,Location#229] csv

== Analyzed Logical Plan ==
OFFENSE_CODE: int, count: bigint
Filter (OFFENSE_CODE#214 = 1402)
+- Aggregate [OFFENSE_CODE#214], [OFFENSE_CODE#214, count(1) AS count#395L]
   +- Relation[INCIDENT_NUMBER#213,OFFENSE_CODE#214,OFFENSE_CODE_GROUP#215,OFFENSE_DESCRIPTION#216,DISTRICT#217,REPORTING_AREA#218,SHOOTING#219,OCCURRED_ON_DATE#220,YEAR#221,MONTH#222,DAY_OF_WEEK#223,HOUR#224,UCR_PART#225,STREET#226,Lat#227,Long#228,Location#229] csv

== Optimized Logical Plan ==
Aggregate [OFFENSE_CODE#214], [OFFENSE_CODE#214, count(1) AS count#395L]
+- Project [OFFENSE_CODE#214]
   +- Filter (isnotnull(OFF

import spark.implicits._



In [12]:
val offenseCodes = spark.read.option("header", "true").option("inferSchema", "true").csv("offense_codes.csv")
offenseCodes.show(100, false)

csv at cmd11.sc:1

1 / 1

csv at cmd11.sc:1

1 / 1

show at cmd11.sc:2

1 / 1

+----+----------------------------------------------------------+
|CODE|NAME                                                      |
+----+----------------------------------------------------------+
|612 |LARCENY PURSE SNATCH - NO FORCE                           |
|613 |LARCENY SHOPLIFTING                                       |
|615 |LARCENY THEFT OF MV PARTS & ACCESSORIES                   |
|1731|INCEST                                                    |
|3111|LICENSE PREMISE VIOLATION                                 |
|2646|LIQUOR - DRINKING IN PUBLIC                               |
|2204|LIQUOR LAW VIOLATION                                      |
|3810|M/V ACCIDENT - INVOLVING �BICYCLE - INJURY                |
|3801|M/V ACCIDENT - OTHER                                      |
|3807|M/V ACCIDENT - OTHER CITY VEHICLE                         |
|3803|M/V ACCIDENT - PERSONAL INJURY                            |
|3805|M/V ACCIDENT - POLICE VEHICLE                             |
|3802|M/V 

offenseCodes: DataFrame = [CODE: int, NAME: string]

In [14]:
import spark.implicits._

val robberyStats = crimeFacts
    .join(offenseCodes, $"CODE" === $"OFFENSE_CODE")
    .filter($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    
robberyStats.show(false)
// robberyStats.explain(true)

show at cmd13.sc:10

12 / 12

show at cmd13.sc:10

1 / 1

show at cmd13.sc:10

200 / 200

show at cmd13.sc:10

200 / 200

+----------------------------------------------+-----+
|NAME                                          |count|
+----------------------------------------------+-----+
|ROBBERY - STREET                              |3056 |
|ROBBERY - FIREARM - BANK                      |3056 |
|ROBBERY - OTHER                               |635  |
|ROBBERY ATTEMPT - OTHER WEAPON - MISCELLANEOUS|635  |
|ROBBERY - KNIFE - CHAIN STORE                 |554  |
|ROBBERY - COMMERCIAL                          |554  |
|ROBBERY ATTEMPT - KNIFE - CHAIN STORE         |181  |
|ROBBERY - BANK                                |181  |
|ROBBERY - HOME INVASION                       |104  |
|ROBBERY - CAR JACKING                         |86   |
|ROBBERY - UNARMED - STREET                    |3    |
|ROBBERY ATTEMPT - KNIFE - BANK                |1    |
|ROBBERY - UNARMED - BUSINESS                  |1    |
|ROBBERY - UNARMED - CHAIN STORE               |1    |
|ROBBERY - KNIFE - STREET                      |1    |
|ROBBERY -

import spark.implicits._


robberyStats: Dataset[Row] = [NAME: string, count: bigint]

In [16]:
import org.apache.spark.sql.functions.broadcast

val offenseCodesBroadcast = broadcast(offenseCodes)

val robberyStatsWithBroadcast = crimeFacts
    .join(offenseCodesBroadcast, $"CODE" === $"OFFENSE_CODE")
    .filter($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    
robberyStatsWithBroadcast.explain(true)

== Parsed Logical Plan ==
'Sort ['count DESC NULLS LAST], true
+- Aggregate [NAME#529], [NAME#529, count(1) AS count#885L]
   +- Filter StartsWith(NAME#529, ROBBERY)
      +- Join Inner, (CODE#528 = OFFENSE_CODE#416)
         :- Relation[INCIDENT_NUMBER#415,OFFENSE_CODE#416,OFFENSE_CODE_GROUP#417,OFFENSE_DESCRIPTION#418,DISTRICT#419,REPORTING_AREA#420,SHOOTING#421,OCCURRED_ON_DATE#422,YEAR#423,MONTH#424,DAY_OF_WEEK#425,HOUR#426,UCR_PART#427,STREET#428,Lat#429,Long#430,Location#431] csv
         +- ResolvedHint (broadcast)
            +- Relation[CODE#528,NAME#529] csv

== Analyzed Logical Plan ==
NAME: string, count: bigint
Sort [count#885L DESC NULLS LAST], true
+- Aggregate [NAME#529], [NAME#529, count(1) AS count#885L]
   +- Filter StartsWith(NAME#529, ROBBERY)
      +- Join Inner, (CODE#528 = OFFENSE_CODE#416)
         :- Relation[INCIDENT_NUMBER#415,OFFENSE_CODE#416,OFFENSE_CODE_GROUP#417,OFFENSE_DESCRIPTION#418,DISTRICT#419,REPORTING_AREA#420,SHOOTING#421,OCCURRED_ON_DATE#422,YEA

import org.apache.spark.sql.functions.broadcast


offenseCodesBroadcast: Dataset[Row] = [CODE: int, NAME: string]
robberyStatsWithBroadcast: Dataset[Row] = [NAME: string, count: bigint]

In [22]:
val cachedCrimeFacts = crimeFacts.select("INCIDENT_NUMBER").cache()

cachedCrimeFacts.count()
cachedCrimeFacts.count()

cachedCrimeFacts.unpersist()

count at cmd21.sc:3

12 / 12

count at cmd21.sc:3

1 / 1

count at cmd21.sc:4

12 / 12

count at cmd21.sc:4

1 / 1

cachedCrimeFacts: Dataset[Row] = [INCIDENT_NUMBER: string]
res21_1: Long = 319073L
res21_2: Long = 319073L
res21_3: Dataset[Row] = [INCIDENT_NUMBER: string]

In [19]:
class NotSerializableClass(value: Int) {
    def getValue = value
}

val breakIt = new NotSerializableClass(1)

sc.parallelize(Seq(1, 2, 3))
    .map { x =>
        x + breakIt.getValue
    }
    .collect()

collect at cmd18.sc:8

12 / 12

defined class NotSerializableClass
res18_1: Array[Int] = Array(2, 3, 4)

In [17]:
import spark.implicits._

val codesTotal = offenseCodes
    .filter('NAME.startsWith("ROBBERY"))
    .select('CODE)
    .distinct
    .count()

val bf = offenseCodes
    .filter('NAME.startsWith("ROBBERY"))
    .stat.bloomFilter('CODE, codesTotal, 0.01)

import org.apache.spark.sql.functions.udf

val mightContain = udf((s: Int) => bf.mightContain(s))

// offenseCodes.filter('NAME.startsWith("ROBBERY")).show(1000)
// offenseCodes.filter(mightContain('CODE)).show(1000)

val robberyStatsWithBf = crimeFacts
    .filter(mightContain('OFFENSE_CODE))
    .join(offenseCodes, 'CODE === 'OFFENSE_CODE)
    .filter('NAME.startsWith("ROBBERY"))
    .groupBy('NAME)
    .count()
    .orderBy('count.desc)

// robberyStatsWithBf.explain(true)
robberyStatsWithBf.show(true)

count at cmd16.sc:5

1 / 1

count at cmd16.sc:5

200 / 200

count at cmd16.sc:5

...

200 / 200

show at cmd16.sc:29

200 / 200

+--------------------+-----+
|                NAME|count|
+--------------------+-----+
|    ROBBERY - STREET| 3056|
|ROBBERY - FIREARM...| 3056|
|ROBBERY ATTEMPT -...|  635|
|     ROBBERY - OTHER|  635|
|ROBBERY - COMMERCIAL|  554|
|ROBBERY - KNIFE -...|  554|
|      ROBBERY - BANK|  181|
|ROBBERY ATTEMPT -...|  181|
|ROBBERY - HOME IN...|  104|
|ROBBERY - CAR JAC...|   86|
|ROBBERY - UNARMED...|    3|
|ROBBERY - KNIFE -...|    1|
|ROBBERY - UNARMED...|    1|
|ROBBERY ATTEMPT -...|    1|
|ROBBERY - UNARMED...|    1|
|ROBBERY - UNARMED...|    1|
+--------------------+-----+



import spark.implicits._


codesTotal: Long = 58L
bf: org.apache.spark.util.sketch.BloomFilter = org.apache.spark.util.sketch.BloomFilterImpl@bd441056
import org.apache.spark.sql.functions.udf


mightContain: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd16$Helper$$Lambda$6508/579758888@1402542,
  BooleanType,
  Some(List(IntegerType))
)
robberyStatsWithBf: Dataset[Row] = [NAME: string, count: bigint]

In [ ]:
robberyStats.explain(true)

In [ ]:
crimeFacts.show

In [ ]:
crimeFacts.select($"year", $"month").distinct.count

In [ ]:
class NotSerializable(v: Int) {
    def getV = v
}

val ns = new NotSerializable(1)

sc.parallelize(Array(1, 2, 3)).map { x =>
    // val ns = new NotSerializable(x)
    ns.getV
}.collect()

In [3]:
crimeFacts.show

show at cmd2.sc:1

1 / 1

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One|       LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|           VANDALISM|     C11|   

In [6]:
crimeFacts.selectExpr("DISTRICT", "percentile_approx(Lat, 0.5) over (partition by DISTRICT) as medianLat").show

show at cmd5.sc:1

12 / 12

show at cmd5.sc:1

1 / 1

show at cmd5.sc:1

4 / 4

+--------+-----------+
|DISTRICT|  medianLat|
+--------+-----------+
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
|      C6|42.33248496|
+--------+-----------+
only showing top 20 rows



In [9]:
crimeFacts.createOrReplaceTempView("crimeFacts")
spark.sql("select DISTRICT, percentile_approx(Lat, 0.5) as medianLat from crimeFacts group by DISTRICT").show

show at cmd8.sc:2

12 / 12

show at cmd8.sc:2

1 / 1

show at cmd8.sc:2

...

show at cmd8.sc:2

100 / 100

show at cmd8.sc:2

75 / 75

+--------+-----------+
|DISTRICT|  medianLat|
+--------+-----------+
|      C6|42.33248496|
|    null| 42.2934621|
|      B2|42.32148976|
|     C11| 42.3005201|
|     E13|42.31538436|
|      B3|42.28560167|
|      E5|42.28365951|
|     A15| 42.3776959|
|      A7|42.37756074|
|     D14|42.35097948|
|      D4|42.34428082|
|     E18|42.26201849|
|      A1|42.35650174|
+--------+-----------+



In [27]:
val colList = crimeFacts.columns

val req = s"""select ${colList.mkString(", ")} from target"""

req

colList: Array[String] = Array(
  "INCIDENT_NUMBER",
  "OFFENSE_CODE",
  "OFFENSE_CODE_GROUP",
  "OFFENSE_DESCRIPTION",
  "DISTRICT",
  "REPORTING_AREA",
  "SHOOTING",
  "OCCURRED_ON_DATE",
  "YEAR",
  "MONTH",
  "DAY_OF_WEEK",
  "HOUR",
  "UCR_PART",
  "STREET",
  "Lat",
  "Long",
  "Location"
)
req: String = """select INCIDENT_NUMBER, OFFENSE_CODE, OFFENSE_CODE_GROUP, OFFENSE_DESCRIPTION, DISTRICT, REPORTING_AREA, SHOOTING, OCCURRED_ON_DATE, YEAR, MONTH, DAY_OF_WEEK, HOUR, UCR_PART, STREET, Lat, Long, Location 
 from target"""
res26_2: String = """select INCIDENT_NUMBER, OFFENSE_CODE, OFFENSE_CODE_GROUP, OFFENSE_DESCRIPTION, DISTRICT, REPORTING_AREA, SHOOTING, OCCURRED_ON_DATE, YEAR, MONTH, DAY_OF_WEEK, HOUR, UCR_PART, STREET, Lat, Long, Location 
 from target"""

In [27]:
val colsForTarget = List("id", "new_id") ++ colList

crimeFacts.select(colsForTarget: _*)

cmd27.sc:3: no `: _*' annotation allowed here
(such annotations are only allowed in arguments to *-parameters)
val res27_1 = crimeFacts.select(colsForTarget: _*)
                                             ^Compilation Failed

: 